In [14]:
import numpy as np
import pickle

def multi_key_dict_get(d, k):
    for keys, v in d.items():
        if k in keys:
            return v
    return None

#64卦、4096種卦爻組合資料庫，爻由底(左)至上(右)起
sixtyfour_gua_list = pickle.load( open( "data/sixtyfour_gua_list.pkl", "rb" ) )
bagua =  pickle.load( open( "data/bagua.pkl", "rb" ) )
def bookgua(): #由底至上起爻
    shifa_results = []
    for i in range(6):
        n=2
        #掛一
        stalks_first = 50-1
        #一變 (分二、掛一、揲四、歸奇)
        first_division = np.random.multinomial(stalks_first, np.ones(n)/2, size=1)[0]
        guayi = 1
        right = first_division[0] - guayi
        left_extract = first_division[1] % 4 
        if left_extract == 0:
            left_extract = 4
        right_extract = right % 4
        if right_extract == 0:
            right_extract = 4
        yibian  = left_extract + right_extract + guayi
        #二變 (分二、掛一、揲四、歸奇)
        stalks_second = stalks_first - yibian
        second_division = np.random.multinomial(stalks_second, np.ones(n)/2, size=1)[0]
        right_second = second_division[0] - guayi
        left_extract_second = second_division[1] % 4 
        if left_extract_second == 0:
            left_extract_second = 4
        right_extract_second = right_second % 4 
        if right_extract_second == 0:
            right_extract_second = 4
        erbian = left_extract_second + right_extract_second + guayi
        #三變 (分二、掛一、揲四、歸奇)
        stalks_third = stalks_second - erbian
        third_division = np.random.multinomial(stalks_third, np.ones(n)/2, size=1)[0]
        right_third = third_division[0] - guayi
        left_extract_third = third_division[1] % 4
        if left_extract_third  == 0:
            left_extract_third = 4
        right_extract_third = right_third % 4 
        if right_extract_third == 0:
            right_extract_third = 4
        sanbian = left_extract_third + right_extract_third + guayi
        yao = int((stalks_first - yibian - erbian - sanbian) / 4)
        shifa_results.append(yao)
    return str(shifa_results[0]) + str(shifa_results[1]) + str(shifa_results[2]) + str(shifa_results[3]) + str(shifa_results[4]) + str(shifa_results[5])

def bookgua_details():
    guayao = bookgua()
    getgua = multi_key_dict_get(sixtyfour_gua_list, guayao)
    yao_results = bagua.get(getgua)
    bian_yao = guayao.replace("6","1").replace("9","1").replace("7","0").replace("8","0")
    dong_yao = bian_yao.count("1")
    explain = "動爻有【"+str(dong_yao )+"】根。"
    dong_yao_change = guayao.replace("6","7").replace("9","8")
    dong_yao_bian_gua = multi_key_dict_get(sixtyfour_gua_list, dong_yao_change)
    g_gua_result = bagua.get(dong_yao_bian_gua)
    g_gua = "【"+getgua+"之"+dong_yao_bian_gua+"】"
    top_bian_yao = bian_yao.rfind("1")+int(1)
    second_bian_yao = bian_yao.rfind("1",0, bian_yao.rfind("1"))+int(1)
    top_jing_yao = bian_yao.rfind("0") + int(1)
    second_jing_yao = bian_yao.rfind("0", 0, bian_yao.rfind("0"))+int(1)
    top = yao_results.get(top_bian_yao)
    second = yao_results.get(second_bian_yao)
    top_2 = yao_results.get(top_jing_yao)
    second_2 = yao_results.get(second_jing_yao)
    explaination2 = None
    try:
        if dong_yao == 0:
            explaination2 = "主要看【"+getgua+"】卦卦辭。",  yao_results[0]
        elif dong_yao == 1: 
            explaination2 = g_gua, "主要看【"+top[0]+top[1]+"】",  top
        elif dong_yao == 2:
            explaination2 = g_gua, "主要看【"+top[0]+top[1]+"】，其次看【"+second[0]+second[1]+"】。", top, second
        elif dong_yao == 3:
            if guayao[0] == '7' or '8':
                explaination2 = g_gua, "【"+getgua+"】卦為貞(我方)，【"+dong_yao_bian_gua+"】卦為悔(他方)。主貞【"+getgua+"】卦", yao_results[0], g_gua_result[0]
            elif guayao[0] == '6' or '9':
                explaination2 = g_gua, "【"+getgua+"】卦為貞(我方)，【"+getgua+"】卦為悔(他方)。主悔【"+dong_yao_bian_gua+"】卦", g_gua_result[0], yao_results[0]
        elif dong_yao == 4:
            explaination2 = g_gua, "主要看【"+getgua+"】的"+top_2[0]+top_2[1]+"，其次看【"+second_2[0]+second_2[1]+"】。", top_2, second_2
        elif dong_yao == 5:    
            explaination2 = g_gua, "主要看【"+top_2[0]+top_2[1]+"】。",  top_2
        elif dong_yao == 6:
            explaination2 = g_gua, "主要看【"+dong_yao_bian_gua+"】卦。", g_gua_result[0]
    except (TypeError, UnboundLocalError):
        pass
    return  guayao, getgua, yao_results, explain, explaination2

In [15]:
alist=[]
for i in range(5000):
    a= bookgua_details()
    alist.append(a)
alist

[('878878',
  '坎',
  {0: '習坎，有孚，維心亨，行有尚。',
   1: '初六：習坎，入於坎窞，凶。',
   2: '九二：坎有險，求小得。',
   3: '六三：來之坎坎，險且枕，入於坎窞，勿用。',
   4: '六四：樽酒簋貳，用缶，納約自牖，終無咎。',
   5: '九五：坎不盈，只既平，無咎。',
   6: '上六：係用徽纆，置於叢棘，三歲不得，凶。'},
  '動爻有【0】根。',
  ('主要看【坎】卦卦辭。', '習坎，有孚，維心亨，行有尚。')),
 ('877897',
  '巽',
  {0: '小亨，利攸往，利見大人。',
   1: '初六：進退，利武人之貞。',
   2: '九二：巽在床下，用史巫紛若，吉無咎。',
   3: '九三：頻巽，吝。',
   4: '六四：悔亡，田獲三品。',
   5: '九五：貞吉悔亡，無不利。 無初有終，先庚三日，後庚三日，吉。',
   6: '上九：巽在床下，喪其資斧，貞凶。'},
  '動爻有【1】根。',
  ('【巽之蠱】', '主要看【九五】', '九五：貞吉悔亡，無不利。 無初有終，先庚三日，後庚三日，吉。')),
 ('888778',
  '萃',
  {0: '亨。 王假有廟，利見大人，亨，利貞。 用大牲吉，利有攸往。',
   1: '初六：有孚不終，乃亂乃萃，若號一握為笑，勿恤，往無咎。',
   2: '六二：引吉，無咎，孚乃利用禴。',
   3: '六三：萃如，嗟如，無攸利，往無咎，小吝。',
   4: '九四：大吉，無咎。',
   5: '九五：萃有位，無咎。 匪孚，元永貞，悔亡。',
   6: '上六：齎咨涕洟，無咎。'},
  '動爻有【0】根。',
  ('主要看【萃】卦卦辭。', '亨。 王假有廟，利見大人，亨，利貞。 用大牲吉，利有攸往。')),
 ('879887',
  '蠱',
  {0: '元亨，利涉大川。 先甲三日，後甲三日。',
   1: '初六：幹父之蠱，有子考無咎，厲終吉。',
   2: '九二：幹母之蠱，不可貞。',
   3: '九三：幹父之蠱，小有悔，無大咎。',
   4: '六四：裕父之蠱，往見吝。',
   5: '六五：幹父之蠱，用譽。',
   6: '上九：不事王侯，高尚其事。'}

In [9]:
sixtyfour_gua_list.get('779977')